In [2]:
# 抽取式阅读理解任务
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, DefaultDataCollator, Trainer, TrainingArguments
from datasets import DatasetDict

In [5]:
datasets = DatasetDict.load_from_disk('../../NLP-data/10-question_answering/mrc_data')
datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'context', 'question', 'answers'],
        num_rows: 10142
    })
    validation: Dataset({
        features: ['id', 'context', 'question', 'answers'],
        num_rows: 3219
    })
    test: Dataset({
        features: ['id', 'context', 'question', 'answers'],
        num_rows: 1002
    })
})

In [3]:
datasets["train"][0]

{'id': 'TRAIN_186_QUERY_0',
 'context': '范廷颂枢机（，），圣名保禄·若瑟（），是越南罗马天主教枢机。1963年被任为主教；1990年被擢升为天主教河内总教区宗座署理；1994年被擢升为总主教，同年年底被擢升为枢机；2009年2月离世。范廷颂于1919年6月15日在越南宁平省天主教发艳教区出生；童年时接受良好教育后，被一位越南神父带到河内继续其学业。范廷颂于1940年在河内大修道院完成神学学业。范廷颂于1949年6月6日在河内的主教座堂晋铎；及后被派到圣女小德兰孤儿院服务。1950年代，范廷颂在河内堂区创建移民接待中心以收容到河内避战的难民。1954年，法越战争结束，越南民主共和国建都河内，当时很多天主教神职人员逃至越南的南方，但范廷颂仍然留在河内。翌年管理圣若望小修院；惟在1960年因捍卫修院的自由、自治及拒绝政府在修院设政治课的要求而被捕。1963年4月5日，教宗任命范廷颂为天主教北宁教区主教，同年8月15日就任；其牧铭为「我信天主的爱」。由于范廷颂被越南政府软禁差不多30年，因此他无法到所属堂区进行牧灵工作而专注研读等工作。范廷颂除了面对战争、贫困、被当局迫害天主教会等问题外，也秘密恢复修院、创建女修会团体等。1990年，教宗若望保禄二世在同年6月18日擢升范廷颂为天主教河内总教区宗座署理以填补该教区总主教的空缺。1994年3月23日，范廷颂被教宗若望保禄二世擢升为天主教河内总教区总主教并兼天主教谅山教区宗座署理；同年11月26日，若望保禄二世擢升范廷颂为枢机。范廷颂在1995年至2001年期间出任天主教越南主教团主席。2003年4月26日，教宗若望保禄二世任命天主教谅山教区兼天主教高平教区吴光杰主教为天主教河内总教区署理主教；及至2005年2月19日，范廷颂因获批辞去总主教职务而荣休；吴光杰同日真除天主教河内总教区总主教职务。范廷颂于2009年2月22日清晨在河内离世，享年89岁；其葬礼于同月26日上午在天主教河内总教区总主教座堂举行。',
 'question': '范廷颂是什么时候被任为主教的？',
 'answers': {'text': ['1963年'], 'answer_start': [30]}}

In [4]:
tokenizer = AutoTokenizer.from_pretrained("hfl/chinese-macbert-base")
tokenizer

'(MaxRetryError("HTTPSConnectionPool(host='huggingface.co', port=443): Max retries exceeded with url: /hfl/chinese-macbert-base/resolve/main/tokenizer_config.json (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x000001A1F47437D0>, 'Connection to huggingface.co timed out. (connect timeout=10)'))"), '(Request ID: 7591c6e1-1de7-48e0-bafe-5eb92db2c7dd)')' thrown while requesting HEAD https://huggingface.co/hfl/chinese-macbert-base/resolve/main/tokenizer_config.json
Using bos_token, but it is not set yet.
Using eos_token, but it is not set yet.


BertTokenizerFast(name_or_path='hfl/chinese-macbert-base', vocab_size=21128, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [5]:
# 取出10条作为例子
sample_dataset = datasets["train"].select(range(10))
sample_dataset

Dataset({
    features: ['id', 'context', 'question', 'answers'],
    num_rows: 10
})

In [6]:
sample_dataset[0]

{'id': 'TRAIN_186_QUERY_0',
 'context': '范廷颂枢机（，），圣名保禄·若瑟（），是越南罗马天主教枢机。1963年被任为主教；1990年被擢升为天主教河内总教区宗座署理；1994年被擢升为总主教，同年年底被擢升为枢机；2009年2月离世。范廷颂于1919年6月15日在越南宁平省天主教发艳教区出生；童年时接受良好教育后，被一位越南神父带到河内继续其学业。范廷颂于1940年在河内大修道院完成神学学业。范廷颂于1949年6月6日在河内的主教座堂晋铎；及后被派到圣女小德兰孤儿院服务。1950年代，范廷颂在河内堂区创建移民接待中心以收容到河内避战的难民。1954年，法越战争结束，越南民主共和国建都河内，当时很多天主教神职人员逃至越南的南方，但范廷颂仍然留在河内。翌年管理圣若望小修院；惟在1960年因捍卫修院的自由、自治及拒绝政府在修院设政治课的要求而被捕。1963年4月5日，教宗任命范廷颂为天主教北宁教区主教，同年8月15日就任；其牧铭为「我信天主的爱」。由于范廷颂被越南政府软禁差不多30年，因此他无法到所属堂区进行牧灵工作而专注研读等工作。范廷颂除了面对战争、贫困、被当局迫害天主教会等问题外，也秘密恢复修院、创建女修会团体等。1990年，教宗若望保禄二世在同年6月18日擢升范廷颂为天主教河内总教区宗座署理以填补该教区总主教的空缺。1994年3月23日，范廷颂被教宗若望保禄二世擢升为天主教河内总教区总主教并兼天主教谅山教区宗座署理；同年11月26日，若望保禄二世擢升范廷颂为枢机。范廷颂在1995年至2001年期间出任天主教越南主教团主席。2003年4月26日，教宗若望保禄二世任命天主教谅山教区兼天主教高平教区吴光杰主教为天主教河内总教区署理主教；及至2005年2月19日，范廷颂因获批辞去总主教职务而荣休；吴光杰同日真除天主教河内总教区总主教职务。范廷颂于2009年2月22日清晨在河内离世，享年89岁；其葬礼于同月26日上午在天主教河内总教区总主教座堂举行。',
 'question': '范廷颂是什么时候被任为主教的？',
 'answers': {'text': ['1963年'], 'answer_start': [30]}}

In [7]:
# stride一般设置成答案token的长度
tokenized_examples = tokenizer(text=sample_dataset["question"],
                               text_pair=sample_dataset["context"],
                               return_offsets_mapping=True,
                               return_overflowing_tokens=True,
                               stride=128,
                               max_length=384, truncation="only_second", padding="max_length")
tokenized_examples.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'offset_mapping', 'overflow_to_sample_mapping'])

In [8]:
# 10条数据被切分成了25条
tokenized_examples["overflow_to_sample_mapping"], len(tokenized_examples["overflow_to_sample_mapping"])

([0,
  0,
  0,
  1,
  1,
  1,
  2,
  2,
  2,
  3,
  3,
  3,
  4,
  4,
  4,
  5,
  5,
  5,
  6,
  6,
  6,
  7,
  7,
  7,
  8,
  8,
  8,
  9,
  9],
 29)

In [9]:
for sen in tokenizer.batch_decode(tokenized_examples["input_ids"][:3]):
    print(sen)

[CLS] 范 廷 颂 是 什 么 时 候 被 任 为 主 教 的 ？ [SEP] 范 廷 颂 枢 机 （ ， ） ， 圣 名 保 禄 · 若 瑟 （ ） ， 是 越 南 罗 马 天 主 教 枢 机 。 1963 年 被 任 为 主 教 ； 1990 年 被 擢 升 为 天 主 教 河 内 总 教 区 宗 座 署 理 ； 1994 年 被 擢 升 为 总 主 教 ， 同 年 年 底 被 擢 升 为 枢 机 ； 2009 年 2 月 离 世 。 范 廷 颂 于 1919 年 6 月 15 日 在 越 南 宁 平 省 天 主 教 发 艳 教 区 出 生 ； 童 年 时 接 受 良 好 教 育 后 ， 被 一 位 越 南 神 父 带 到 河 内 继 续 其 学 业 。 范 廷 颂 于 1940 年 在 河 内 大 修 道 院 完 成 神 学 学 业 。 范 廷 颂 于 1949 年 6 月 6 日 在 河 内 的 主 教 座 堂 晋 铎 ； 及 后 被 派 到 圣 女 小 德 兰 孤 儿 院 服 务 。 1950 年 代 ， 范 廷 颂 在 河 内 堂 区 创 建 移 民 接 待 中 心 以 收 容 到 河 内 避 战 的 难 民 。 1954 年 ， 法 越 战 争 结 束 ， 越 南 民 主 共 和 国 建 都 河 内 ， 当 时 很 多 天 主 教 神 职 人 员 逃 至 越 南 的 南 方 ， 但 范 廷 颂 仍 然 留 在 河 内 。 翌 年 管 理 圣 若 望 小 修 院 ； 惟 在 1960 年 因 捍 卫 修 院 的 自 由 、 自 治 及 拒 绝 政 府 在 修 院 设 政 治 课 的 要 求 而 被 捕 。 1963 年 4 月 5 日 ， 教 宗 任 命 范 廷 颂 为 天 主 教 北 宁 教 区 主 教 ， 同 年 8 月 15 日 就 任 ； 其 牧 铭 为 「 我 信 [SEP]
[CLS] 范 廷 颂 是 什 么 时 候 被 任 为 主 教 的 ？ [SEP] 越 南 民 主 共 和 国 建 都 河 内 ， 当 时 很 多 天 主 教 神 职 人 员 逃 至 越 南 的 南 方 ， 但 范 廷 颂 仍 然 留 在 河 内 。 翌 年 管 理 圣 若 望 小 修 院 ； 惟 在 1960 年 因 捍 卫 修 院 的 自 由 、 自 治 及 拒 

In [10]:
tokenized_examples["offset_mapping"][:3]

[[(0, 0),
  (0, 1),
  (1, 2),
  (2, 3),
  (3, 4),
  (4, 5),
  (5, 6),
  (6, 7),
  (7, 8),
  (8, 9),
  (9, 10),
  (10, 11),
  (11, 12),
  (12, 13),
  (13, 14),
  (14, 15),
  (0, 0),
  (0, 1),
  (1, 2),
  (2, 3),
  (3, 4),
  (4, 5),
  (5, 6),
  (6, 7),
  (7, 8),
  (8, 9),
  (9, 10),
  (10, 11),
  (11, 12),
  (12, 13),
  (13, 14),
  (14, 15),
  (15, 16),
  (16, 17),
  (17, 18),
  (18, 19),
  (19, 20),
  (20, 21),
  (21, 22),
  (22, 23),
  (23, 24),
  (24, 25),
  (25, 26),
  (26, 27),
  (27, 28),
  (28, 29),
  (29, 30),
  (30, 34),
  (34, 35),
  (35, 36),
  (36, 37),
  (37, 38),
  (38, 39),
  (39, 40),
  (40, 41),
  (41, 45),
  (45, 46),
  (46, 47),
  (47, 48),
  (48, 49),
  (49, 50),
  (50, 51),
  (51, 52),
  (52, 53),
  (53, 54),
  (54, 55),
  (55, 56),
  (56, 57),
  (57, 58),
  (58, 59),
  (59, 60),
  (60, 61),
  (61, 62),
  (62, 63),
  (63, 67),
  (67, 68),
  (68, 69),
  (69, 70),
  (70, 71),
  (71, 72),
  (72, 73),
  (73, 74),
  (74, 75),
  (75, 76),
  (76, 77),
  (77, 78),
  (78, 79)

In [11]:
# offset_mapping长度和tokens的数量一致，表示从原始文本中(start, end)映射到tokens中(start, end)的位置
print(tokenized_examples["offset_mapping"][0], len(tokenized_examples["offset_mapping"][0]))


[(0, 0), (0, 1), (1, 2), (2, 3), (3, 4), (4, 5), (5, 6), (6, 7), (7, 8), (8, 9), (9, 10), (10, 11), (11, 12), (12, 13), (13, 14), (14, 15), (0, 0), (0, 1), (1, 2), (2, 3), (3, 4), (4, 5), (5, 6), (6, 7), (7, 8), (8, 9), (9, 10), (10, 11), (11, 12), (12, 13), (13, 14), (14, 15), (15, 16), (16, 17), (17, 18), (18, 19), (19, 20), (20, 21), (21, 22), (22, 23), (23, 24), (24, 25), (25, 26), (26, 27), (27, 28), (28, 29), (29, 30), (30, 34), (34, 35), (35, 36), (36, 37), (37, 38), (38, 39), (39, 40), (40, 41), (41, 45), (45, 46), (46, 47), (47, 48), (48, 49), (49, 50), (50, 51), (51, 52), (52, 53), (53, 54), (54, 55), (55, 56), (56, 57), (57, 58), (58, 59), (59, 60), (60, 61), (61, 62), (62, 63), (63, 67), (67, 68), (68, 69), (69, 70), (70, 71), (71, 72), (72, 73), (73, 74), (74, 75), (75, 76), (76, 77), (77, 78), (78, 79), (79, 80), (80, 81), (81, 82), (82, 83), (83, 84), (84, 85), (85, 86), (86, 87), (87, 91), (91, 92), (92, 93), (93, 94), (94, 95), (95, 96), (96, 97), (97, 98), (98, 99), (

In [12]:
sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")

In [13]:
sample_mapping

[0,
 0,
 0,
 1,
 1,
 1,
 2,
 2,
 2,
 3,
 3,
 3,
 4,
 4,
 4,
 5,
 5,
 5,
 6,
 6,
 6,
 7,
 7,
 7,
 8,
 8,
 8,
 9,
 9]

In [14]:
offset_mapping = tokenized_examples.pop("offset_mapping")

In [15]:

# 遍历10调数据中的offset_mapping
for idx, _ in enumerate(sample_mapping):
    # 取出数据集中的一条数据
    answer = sample_dataset[sample_mapping[idx]]["answers"]
    # 获得原始答案文本中的起始和结束位置
    char_ori_start = answer["answer_start"][0]
    char_ori_end = char_ori_start + len(answer["text"][0])
    # 获得token后的context的起始和结束位置
    context_token_start = tokenized_examples.sequence_ids(idx).index(1)
    context_token_end = tokenized_examples.sequence_ids(idx).index(None, context_token_start) - 1
    print(answer, char_ori_start, char_ori_end, context_token_start, context_token_end)
    offset = offset_mapping[idx]
    # 寻找answer在token后的序列中的起始位置和结束位置
    # 判断答案是否在原始的context中
    # 如果context结束位置(右边界)比原始答案的起始位置小
    # 或者context起始位置(左边界)比原始答案的结束位置大
    # 则说明原始答案不在context中
    if offset[context_token_end][1] < char_ori_start and \
    offset[context_token_start][0] > char_ori_end:
        start_pos_token = 0
        end_pos_token = 0
    else:
        # 去寻找answer在token后的序列中的起始位置和结束位置
        token_id = context_token_start
        # 从context_token_start开始遍历，最长到context_token_end结束
        while token_id <= context_token_end and offset[token_id][0] < char_ori_start:
            token_id += 1
        start_pos_token = token_id
        token_id = context_token_end
        while token_id >= context_token_start and offset[token_id][1] > char_ori_end:
            token_id -=1
        end_token_pos = token_id
    print(tokenizer.decode(tokenized_examples["input_ids"][idx][start_pos_token: end_token_pos+1]))


{'text': ['1963年'], 'answer_start': [30]} 30 35 17 382
1963 年
{'text': ['1963年'], 'answer_start': [30]} 30 35 17 382

{'text': ['1963年'], 'answer_start': [30]} 30 35 17 289

{'text': ['1990年被擢升为天主教河内总教区宗座署理'], 'answer_start': [41]} 41 62 15 382
1990 年 被 擢 升 为 天 主 教 河 内 总 教 区 宗 座 署 理
{'text': ['1990年被擢升为天主教河内总教区宗座署理'], 'answer_start': [41]} 41 62 15 382

{'text': ['1990年被擢升为天主教河内总教区宗座署理'], 'answer_start': [41]} 41 62 15 283

{'text': ['范廷颂于1919年6月15日在越南宁平省天主教发艳教区出生'], 'answer_start': [97]} 97 126 15 382
范 廷 颂 于 1919 年 6 月 15 日 在 越 南 宁 平 省 天 主 教 发 艳 教 区 出 生
{'text': ['范廷颂于1919年6月15日在越南宁平省天主教发艳教区出生'], 'answer_start': [97]} 97 126 15 382

{'text': ['范廷颂于1919年6月15日在越南宁平省天主教发艳教区出生'], 'answer_start': [97]} 97 126 15 283

{'text': ['1994年3月23日，范廷颂被教宗若望保禄二世擢升为天主教河内总教区总主教并兼天主教谅山教区宗座署理'], 'answer_start': [548]} 548 598 17 382

{'text': ['1994年3月23日，范廷颂被教宗若望保禄二世擢升为天主教河内总教区总主教并兼天主教谅山教区宗座署理'], 'answer_start': [548]} 548 598 17 382
1994 年 3 月 23 日 ， 范 廷 颂 被 教 宗 若 望 保 禄 二 世 擢 升 为 天 主 教 河 内 总 教 区 总 主 教

In [16]:
"""
sample可以理解成是一个batch的数据
sample可以理解成一个{"id": [xxx, xxx, ...]
                   "question": [xxx, xxx], 
                   "context": [xxx, xxx], 
                   "answers": [xxx, xxx]}的字典
"""
# 
def process_func(sample):
    # tokenized_examples可以理解成一个{"input_ids": [[xxx], [xxx], ...],
    #                   "token_type_ids": [[xxx], [xxx]], 
    #                   "attention_mask":[[xxx], [xxx]]
    #                   "offset_mapping": [[xxx], [xxx]], 
    #                   "overflow_to_sample_mapping": [[xxx], [xxx]}的字典
    tokenized_examples = tokenizer(text=sample["question"],
                                   text_pair=sample["context"],
                                   return_offsets_mapping=True,
                                   return_overflowing_tokens=True,  # 启用滑动窗口
                                   stride=128,  # 滑动窗口重叠token长度
                                   max_length=384, truncation="only_second", padding="max_length")
    # offset_mapping = tokenized_examples.pop("offset_mapping")
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")  # [0, 0, 1, 1, 1, 2, 2, 2, ...]
    # 遍历batch中每条数据的offset_mapping
    start_positions = []
    end_positions = []
    example_ids = []  # 记录数据的唯一标识
    # 遍历滑动窗口后的生成的每一条数据
    for idx, _ in enumerate(sample_mapping):
        # 取出数据集中的一条数据
        answer = sample["answers"][sample_mapping[idx]]
        # 获得原始答案文本中的起始和结束位置 
        char_ori_start = answer["answer_start"][0]
        char_ori_end = char_ori_start + len(answer["text"][0])
        # 获得token后的context的起始和结束位置
        context_token_start = tokenized_examples.sequence_ids(idx).index(1)
        context_token_end = tokenized_examples.sequence_ids(idx).index(None, context_token_start) - 1
        # 寻找answer在token后的序列中的起始位置和结束位置
        # 判断答案是否在原始的context中
        # 如果context结束位置(右边界)比原始答案的起始位置小
        # 或者context起始位置(左边界)比原始答案的结束位置大
        # 则说明原始答案不在context中
        offset = tokenized_examples["offset_mapping"][idx]
        if offset[context_token_end][1] < char_ori_start and \
        offset[context_token_start][0] > char_ori_end:
            start_token_pos= 0
            end_token_pos = 0
        else:
            # 去寻找answer在token后的序列中的起始位置和结束位置
            token_id = context_token_start
            # 从context_token_start开始遍历，最长到context_token_end结束
            # 逐渐逼近原始答案的起始位置
            # token后的数据通过offset_mapping联系原始文本
            while token_id <= context_token_end and offset[token_id][0] < char_ori_start:
                token_id += 1
            start_token_pos = token_id
            token_id = context_token_end
            while token_id >= context_token_start and offset[token_id][1] > char_ori_end:
                token_id -=1
            end_token_pos = token_id
        start_positions.append(start_token_pos)
        end_positions.append(end_token_pos)
        example_ids.append(sample["id"][sample_mapping[idx]])

        # 修改offset_mapping字段，让context处的值保持原值，query处的变成None
        tokenized_examples["offset_mapping"][idx] = [(o if tokenized_examples.sequence_ids(idx)[i] == 1 else None) 
                                                     for i, o in enumerate(tokenized_examples["offset_mapping"][idx])]
       

    tokenized_examples["start_positions"] = start_positions
    tokenized_examples["end_positions"] = end_positions
    tokenized_examples["example_ids"] = example_ids
 
    return tokenized_examples

In [17]:
tokenized_datasets = datasets.map(process_func, batched=True, remove_columns=datasets["train"].column_names)
tokenized_datasets

Map:   0%|          | 0/10142 [00:00<?, ? examples/s]

Map:   0%|          | 0/3219 [00:00<?, ? examples/s]

Map:   0%|          | 0/1002 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'offset_mapping', 'start_positions', 'end_positions', 'example_ids'],
        num_rows: 19189
    })
    validation: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'offset_mapping', 'start_positions', 'end_positions', 'example_ids'],
        num_rows: 6327
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'offset_mapping', 'start_positions', 'end_positions', 'example_ids'],
        num_rows: 1988
    })
})

In [21]:
tokenized_datasets["train"][0]["offset_mapping"]

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 [0, 1],
 [1, 2],
 [2, 3],
 [3, 4],
 [4, 5],
 [5, 6],
 [6, 7],
 [7, 8],
 [8, 9],
 [9, 10],
 [10, 11],
 [11, 12],
 [12, 13],
 [13, 14],
 [14, 15],
 [15, 16],
 [16, 17],
 [17, 18],
 [18, 19],
 [19, 20],
 [20, 21],
 [21, 22],
 [22, 23],
 [23, 24],
 [24, 25],
 [25, 26],
 [26, 27],
 [27, 28],
 [28, 29],
 [29, 30],
 [30, 34],
 [34, 35],
 [35, 36],
 [36, 37],
 [37, 38],
 [38, 39],
 [39, 40],
 [40, 41],
 [41, 45],
 [45, 46],
 [46, 47],
 [47, 48],
 [48, 49],
 [49, 50],
 [50, 51],
 [51, 52],
 [52, 53],
 [53, 54],
 [54, 55],
 [55, 56],
 [56, 57],
 [57, 58],
 [58, 59],
 [59, 60],
 [60, 61],
 [61, 62],
 [62, 63],
 [63, 67],
 [67, 68],
 [68, 69],
 [69, 70],
 [70, 71],
 [71, 72],
 [72, 73],
 [73, 74],
 [74, 75],
 [75, 76],
 [76, 77],
 [77, 78],
 [78, 79],
 [79, 80],
 [80, 81],
 [81, 82],
 [82, 83],
 [83, 84],
 [84, 85],
 [85, 86],
 [86, 87],
 [87, 91],
 [91, 92],
 [92, 93],
 [93, 94]

In [19]:
tokenized_datasets["train"]["example_ids"][:10]

['TRAIN_186_QUERY_0',
 'TRAIN_186_QUERY_0',
 'TRAIN_186_QUERY_0',
 'TRAIN_186_QUERY_1',
 'TRAIN_186_QUERY_1',
 'TRAIN_186_QUERY_1',
 'TRAIN_186_QUERY_2',
 'TRAIN_186_QUERY_2',
 'TRAIN_186_QUERY_2',
 'TRAIN_186_QUERY_3']

In [23]:
"""
由于原始数据集的一条数据会由于滑动窗口的原因，被切割成多条数据，
因此需要构建映射关系，让原始数据和切割后的数据对应起来。
"""
# example和feature的映射
from collections import defaultdict
example_to_feature = defaultdict(list)
for idx, example_id in enumerate(tokenized_datasets["train"]["example_ids"][:10]):
    example_to_feature[example_id].append(idx)
example_to_feature

defaultdict(list,
            {'TRAIN_186_QUERY_0': [0, 1, 2],
             'TRAIN_186_QUERY_1': [3, 4, 5],
             'TRAIN_186_QUERY_2': [6, 7, 8],
             'TRAIN_186_QUERY_3': [9]})

In [ ]:
import numpy as np
def get_result(start_logits, end_logits, examples, tokenized_example):
    predictions = {}
    references = {}
    # 完成example和feature的映射
    example_to_feature = defaultdict(list)
    for idx, example_id in enumerate(tokenized_example["example_ids"]):
        example_to_feature[example_id].append(idx)
    # 最优答案数量
    n_best = 20 
    # 最大答案长度
    max_answer_length = 30
    # examples是一个Dataset
    # 遍历每一条数据
    for example in examples:
        query = example["id"]
        context = example["context"]
        # 记录这条数据的所有答案
        answers = []
        # 通过query找到对应的feature_id
        # 通过feature_id去取出对应的feature信息
        for feature_id in example_to_feature[query]:
            # 获取分数为top20的起始索引(token上) 
            start_logit = start_logits[feature_id]
            start_idxs = np.argsort(start_logit)[::-1][:n_best].tolist()
            # 获得分数为top20的结束索引(token上)
            end_logit = end_logits[feature_id]
            end_idxs = np.argsort(end_logit)[::-1][:n_best].tolist()
            # 取出offset
            offset = tokenized_example["offset_mapping"][feature_id]
            for start_idx in start_idxs:
                for end_idx in end_idxs:
                    if offset[start_idx] is None or offset[end_idx] is None:
                        continue
                    if end_idx < start_idx or end_idx - start_idx + 1 > max_answer_length:
                        continue
                    answers.append({"text": context[offset[start_idx][0]: offset[end_idx][1]],
                                    "score":start_logit[start_idx] + end_logit[end_idx]})
        # 聚合出最终的答案
        if len(answers) > 0:
            best_answer = max(answers, key=lambda x: x["score"])
            predictions[query] = best_answer["text"]
        else:
            predictions[query] = ""
        references[query] = example["answer"]["text"]
    return predictions, references

In [24]:
from cmrc_eval import evaluate_cmrc
def metric(pred):
    start_logits, end_logits = pred[0]
    if start_logits.shape[0] == len(tokenized_datasets["validation"]):
        p, r = get_result(start_logits, end_logits, datasets["validation"], tokenized_datasets["validation"])
    else:
        p, r = get_result(start_logits, end_logits, datasets["test"], tokenized_datasets["test"])
    return evaluate_cmrc(p, r)

In [19]:
model = AutoModelForQuestionAnswering.from_pretrained("hfl/chinese-macbert-base")

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at hfl/chinese-macbert-base and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [25]:
args = TrainingArguments(
    output_dir="models_for_qa",
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    evaluation_strategy="steps",
    eval_steps=200,
    save_strategy="epoch",
    logging_steps=50,
    num_train_epochs=3
)

In [20]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    compute_metrics=metric,
    data_collator=DefaultDataCollator()
)

In [21]:
from transformers import pipeline

pipe = pipeline("question-answering", model=model, tokenizer=tokenizer, device=0)
pipe

In [ ]:
pipe(question="小明在哪里上班？", context="小明在北京上班。")